# Tips:
### To use this app:<br>
1. The address sending the mail must enable the IMAP, see [here](https://support.google.com/mail/answer/7126229?p=BadCredentials&visit_id=637320813303855593-717697157&rd=2#cantsignin). <br>
2. Must allow less secure apps to ON, go [here](https://myaccount.google.com/lesssecureapps).<br>

### References:
[ref. 1](http://wiki.alarmchang.com/index.php?title=Mail_%E7%99%BC%E4%BF%A1_smtp_%E9%9C%80%E8%A6%81%E8%AA%8D%E8%AD%89%EF%BC%8C%E4%B8%94%E9%83%B5%E4%BB%B6%E9%A1%AF%E7%A4%BA_To,_CC,_BCC)<br>
[ref. 2](https://medium.com/jeasee%E9%9A%A8%E7%AD%86/python-email%E5%AF%84%E4%BF%A1-ba2b5eb05d6b)<br>
[ref. 3](https://realpython.com/python-send-email/)<br>
[ref. 4](https://www.itread01.com/p/457779.html)

### Import packages.

In [3]:
import datetime, re, smtplib, requests, os, shutil
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

### Function used to download the file that will be attached.

In [4]:
def APPENDER(message, item_name):
    list_of_item = []
    counter = 0 
    while True:
        print("%s # %d"%(item_name, counter+1))
        item = input(message)
        if item == 'q':
            break
        elif item != '':
            list_of_item.append(item)
            counter += 1
    print("Total %d %s(s) selected."%(counter, item_name))
    return list_of_item

def GET_FILES(file_name, path, url='', original_file_path=''):
    filecontent = []
    if url!='':
        for index, link in enumerate(url):
            r = requests.get(url[index], allow_redirects=True)
            with open('%s/%s'%(path,file_name[index]), 'wb') as f:
                f.write(r.content)
                f.close()
            with open("%s/%s"%(path,file_name[index]), "rb") as f:
                filecontent.append(f.read())
                f.close()
    elif original_file_path!='':
        for index, file_path in enumerate(original_file_path):
#             shutil.copy2(file_path, path+'/'+file_name[index])
#         for file in os.listdir('%s'%path):
#             if 'pdf' in file:
            with open(file_path, "rb") as f:
                filecontent.append(f.read())
                f.close()
    else:
        raise RuntimeError("No url and no file path! No file to process!")
    return filecontent

def DO_ATTACHING():
    url, original_file_path, file_name = '', '', ''
    folder_name = 'Test_Files'   # defaulted, you can change this of course
    path = './%s'%folder_name
    print("============================\nNow attach some files:")
    while True:
        message = input("Download via URL or from existing files? (u/f)")
        if message == 'u':
            url = APPENDER(message="Please enter URL(s), or use \"q\" to quit.", item_name="URL")
            print("============================\nNow name the attachment(s):")
            file_name = APPENDER(message="Please enter file name(s), or use \"q\" to quit.", item_name="File name")
            if len(url) == len(file_name):
                print("Making temporarily folder (%s) at path (./) to save the attachments..."%(folder_name))
                try:
                    os.mkdir(path)
                except:
                    print('Folder already exits!\n')
            else:
                raise RuntimeError("Number of URL(s) does not match number of file names!")
            break
        elif message == 'f':
            original_file_path = APPENDER(message="Please enter the existing file path(s), or use \"q\" to quit.", item_name="Path")
            print("============================\nNow name the attachment(s):")
            file_name = APPENDER(message="Please enter file name(s), or use \"q\" to quit.", item_name="File name")
            break
        else:
            print("*********************Please enter \"u\" or \"f\"!*********************")

    attachment = []
    print("Start attaching attachment(s)...")
    for index, filecontent in enumerate(GET_FILES(file_name, path, url, original_file_path)):
        file = MIMEText(filecontent, "base64", "utf-8")
        file["Content-Type"]="application/octet-stream"   
        file["Content-Disposition"]='attachment; filename=%s'%file_name[index]
        attachment.append(file)
    print("Attaching Ends!")
    return attachment

### Calculate the latest meeting date automatically.

In [5]:
today_weekday = datetime.datetime.now().weekday()
hour = datetime.datetime.now().hour

if today_weekday>4 or (today_weekday==4 and hour>=12):
    how_many_day = 4 - today_weekday + 7
else:
    how_many_day = 4- today_weekday
    
meeting_date = datetime.datetime.fromtimestamp(how_many_day*86400 + datetime.datetime.now().timestamp()).date()

if meeting_date.weekday()!=4:
    print("Automatic meeting date calculation wrong! Please enter the meeting date manually!")
    month = input("Month is:")
    day = input("Day is:")
else:
    month = meeting_date.month; day = meeting_date.day
    
    while True:
        message = input("Meeting date is on %d/%d , do you confirm? (y/n)"%(month, day))
        if message == 'y':
            print("Confirmed.")
            break
        elif message == 'n':
            print("Enter the date manually...")
            month = input("Month is:")
            day = input("Day is:")
            break
        else:
            print("*********************Please enter \"y\" or \"n\"!*********************")

Meeting date is on 8/28 , do you confirm? (y/n)y
Confirmed.


### Create the email list and assign your own email address, which will be used to cc.

In [11]:
email_list = 'ABC <ABC@gmail.com>\
              BCD <BCD@gmail.com>\
              CDE <CDE@gmail.com>\
              DEF <DEF@gmail.com>\
              EFG <EFG@gmail.com>'

email_list = (re.sub('\>[^\<]*','',email_list)).split('<')[1:] #recipients' address
################################################################################
# name_lsit = re.sub('\s','',(re.sub('\<[^\>]*','',email_list))).split('>')[:-1]
# email_name_dict = dict(zip(email_list,name_lsit))
#my_name = email_name_dict[my_addr]
################################################################################

my_addr = input('Enter your email address.')
my_name = input('Enter your english name.')

from_addr = 'XYZ@gmail.com'  # The address where email will be sent from
# msg_text = 'Dear all\nThe paper attached is to be introduced on the coming Friday (%d/%d). Thank you!\n\nSincerely,\n%s'\
#            %(month, day, my_name)    # defaulted, you can change this of course
msg_html = """
            <html>
            <body>
            <strong>Main Text:</strong><br>
            <blockquote>Dear all,</blockquote>
            <blockquote>The paper attached is to be introduced on the coming Friday (%d/%d). Thank you!</blockquote>
            <blockquote>Sincerely,</blockquote>
            <blockquote>%s</blockquote>
            <br>
            <strong>P.S.</strong><br>
            <blockquote>This is a mail send auto-matically by the code  <a href="https://this_is_a_url">here</a>.</blockquote>
            <blockquote>The original purpose is to generate the meeting-email by code; however it is also possible to apply it on Triton system, say sending email by itself if the measurement has ended, etc...</blockquote>
            <blockquote>You are welcome to use or imporve the code if you are interested in it.</blockquote>
            </body>
            </html>
            """%(month, day, my_name)
### This is for test. If you want to use this app, comment this out.

Enter your email address.aggag
Enter your english name.21313


### Attach the paper(s), either by url or by assigning the path. Must make a new directory (defaulted as 'Test_Files' here) to cache the files (might need to enalbe Pulse Secure for downloading due to IP issue).

In [9]:
while True:
    message = input("Is there any file you want to attach? (y/n)")
    if message == 'y':
        attachment = DO_ATTACHING()
        break
    elif message == 'n':
        attachment = []
        break
    else:
        print("*********************Please enter \"y\" or \"n\"!*********************")

Is there any file you want to attach? (y/n)n


### Combine the body text with the attached files.

In [10]:
# body = MIMEText(msg_text, 'plain', 'utf8')
body = MIMEText(msg_html, 'html', 'utf8')
# body.set_charset("utf-8")

msg = MIMEMultipart()
msg["Subject"] = "Group Meeting on %d/%d"%(month,day) #Subject
msg["From"] = from_addr # sending address
msg["To"] = ','.join(email_list) # recipients' address
msg["Cc"] = my_addr # cc
msg.attach(body)

if attachment != []:
    for file in attachment:
        msg.attach(file)

In [44]:
smtp = smtplib.SMTP('smtp.gmail.com',587)
smtp.ehlo()
smtp.starttls()
smtp.login('semi105a@phys.ntu.edu.tw',password='semi105ag')

status = smtp.sendmail(from_addr, email_list, msg.as_string())

if status == {}:
    print("Email sent successfully!")
else:
    print("Email-sending failed!")
# smtp.quit()

Email sent successfully!
